In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import pickle
import warnings
from datetime import timedelta 
import datetime
import lightgbm as lgb

from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from statsmodels.tsa.arima.model import ARIMA
import warnings
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric
from sklearn.metrics import mean_squared_error
import pmdarima

from darts import TimeSeries
from darts.models.forecasting.tft_model import TFTModel
from darts.metrics import mse
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape, mae
from torchmetrics.regression import MeanAbsoluteError

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# -1 Loading Data

In [ ]:
#csv_file_path = 
train_df = pd.read_csv('../raw_data/sales_train_validation.csv')
prices_df = pd.read_csv('../raw_data/sell_prices.csv')
calendar_df = pd.read_csv('../raw_data/calendar.csv')

In [ ]:
train_df_filtered = train_df.iloc[0:5000,:]

In [ ]:
train_df_sample = train_df_filtered.melt(id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'])

In [ ]:
train_df_sample.rename(columns={'variable': 'd', 'value':'sales'},inplace=True)

In [ ]:
merge_df = train_df_sample.merge(calendar_df,on='d',how='left')

In [ ]:
merge_df = merge_df.merge(prices_df,on=['store_id', 'item_id','wm_yr_wk'],how='left')
merge_df

# 0 Importing Data

In [ ]:
# Load your dataset
merge_df['date'] = pd.to_datetime(merge_df['date'])
merge_df.set_index('date', inplace=True)

merge_df

In [ ]:
#FILLING THE EMPTY PLACES
merge_df['sell_price'].fillna(0, inplace=True)
merge_df['event_name_1'].fillna('missing', inplace=True)
merge_df['event_type_1'].fillna('missing', inplace=True)
merge_df['event_name_2'].fillna('missing', inplace=True)
merge_df['event_type_2'].fillna('missing', inplace=True)


In [ ]:
# Assuming df is your DataFrame with datetime index and "id" column
start_date = merge_df.index.min()
end_date = merge_df.index.max()

# Generate the complete date range
complete_date_range = pd.date_range(start=start_date, end=end_date)

# Get unique values of "id" column
unique_ids = merge_df['id'].unique()

merge_df = merge_df.reset_index()

# Create a MultiIndex with Cartesian product of date range and unique ids
multi_index = pd.MultiIndex.from_product([complete_date_range, unique_ids], names=['date', 'id'])

# Reindex the DataFrame using the MultiIndex
merge_df = merge_df.set_index(['date', 'id']).reindex(multi_index)

# Reset the index to make "date" and "id" columns again
merge_df = merge_df.reset_index()

In [ ]:
merge_df.set_index("date",inplace=True)

In [ ]:
# Scale 'sell_price' and 'year' by using MinMaxScaler
minmax_scaler = MinMaxScaler()

merge_df[['sell_price']] = minmax_scaler.fit_transform(merge_df[['sell_price']])
merge_df[['year']] = minmax_scaler.fit_transform(merge_df[['year']])

In [ ]:
# Check unique values for 'cat_id'
print(f"The unique values for 'cat_id' are {merge_df['cat_id'].unique()}")

# Check unique values for 'store_id'
print(f"The unique values for 'store_id' are {merge_df['store_id'].unique()}")

# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)

# Fit encoder for both 'cat_id' and 'store_id'
ohe.fit(merge_df[['cat_id', 'store_id']])

# Display the detected categories for both columns
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names for both columns
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the 'cat_id' and 'store_id' columns
encoded_columns = ohe.transform(merge_df[['cat_id', 'store_id']])

# Drop the original 'cat_id' and 'store_id' columns
merge_df.drop(columns=['cat_id', 'store_id'], inplace=True)

# Concatenate the encoded columns to the DataFrame
merge_df[ ohe.get_feature_names_out()] = encoded_columns


In [ ]:
# Check unique values
print(f"The unique values for 'event_type_1' are {merge_df['event_type_1'].unique()}")

# Fit encoder
ohe.fit(merge_df[['event_type_1']])

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "cat_id" column
merge_df[ohe.get_feature_names_out()] = ohe.transform(merge_df[['event_type_1']])

# Drop the column "cat_id" which has been encoded
merge_df.drop(columns = ['event_type_1'], inplace = True)

In [ ]:
# Check unique values
print(f"The unique values for 'event_type_2' are {merge_df['event_type_2'].unique()}")

# Fit encoder
ohe.fit(merge_df[['event_type_2']])

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "cat_id" column
merge_df[ohe.get_feature_names_out()] = ohe.transform(merge_df[['event_type_2']])

# Drop the column "cat_id" which has been encoded
merge_df.drop(columns = ['event_type_2'], inplace = True)

In [ ]:
# Check unique values
print(f"The unique values for 'event_name_1' are {merge_df['event_name_1'].unique()}")

# Fit encoder
ohe.fit(merge_df[['event_name_1']])

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "cat_id" column
merge_df[ohe.get_feature_names_out()] = ohe.transform(merge_df[['event_name_1']])

# Drop the column "cat_id" which has been encoded
merge_df.drop(columns = ['event_name_1'], inplace = True)

In [ ]:
# Check unique values
print(f"The unique values for 'event_name_2' are {merge_df['event_name_2'].unique()}")

# Fit encoder
ohe.fit(merge_df[['event_name_2']])

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "cat_id" column
merge_df[ohe.get_feature_names_out()] = ohe.transform(merge_df[['event_name_2']])

# Drop the column "cat_id" which has been encoded
merge_df.drop(columns = ['event_name_2'], inplace = True)

In [ ]:
#Encoding Cyclical Features for weekdays
# Notice that Sat starts as 1 till Fri as 7 for 'wday'
merge_df['wday_sin'] = np.sin(2 * np.pi * merge_df['wday'] /7.0)
merge_df['wday_cos'] = np.cos(2 * np.pi * merge_df['wday'] /7.0)

In [ ]:
#Encoding Cyclical Features for month

merge_df['month_sin'] = np.sin(2 * np.pi * merge_df['month'] /12.0)
merge_df['month_cos'] = np.cos(2 * np.pi * merge_df['month'] /12.0)

In [ ]:
merge_df_scaled = merge_df.drop(columns=['d', 'wm_yr_wk','weekday'])

In [ ]:
# Downcast numeric columns
numeric_columns = merge_df_scaled.select_dtypes(include=['int64', 'float64']).columns
merge_df_scaled[numeric_columns] = merge_df_scaled[numeric_columns].apply(lambda x: pd.to_numeric(x, downcast='integer' if np.issubdtype(x.dtype, np.integer) else 'float'))

# Confirm the new datatypes
print(merge_df_scaled.dtypes.unique())

In [ ]:
def feature_extraction(merge_df_scaled):
    # Ignore all warnings within this function
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    
        #lagged features
        for i in range(1, 8):
            merge_df_scaled[f'sales_lag_{i}'] = merge_df_scaled['sales'].shift(i)
    
        #lagged features per years
        for i in range(1, 4):
            merge_df_scaled[f'sales_lag_{i}years'] = merge_df_scaled['sales'].shift(i * 365)
    
            #rolling sum
        merge_df_scaled['rolling_sum_7'] = merge_df_scaled['sales'].rolling(window=7).sum()
        merge_df_scaled['rolling_sum_30'] = merge_df_scaled['sales'].rolling(window=30).sum()
        merge_df_scaled['rolling_sum_60'] = merge_df_scaled['sales'].rolling(window=60).sum()
        merge_df_scaled['rolling_sum_90'] = merge_df_scaled['sales'].rolling(window=90).sum()
        merge_df_scaled['rolling_sum_120'] = merge_df_scaled['sales'].rolling(window=120).sum()
    
        #rolling average
        merge_df_scaled['rolling_mean_7'] = merge_df_scaled['sales'].rolling(window=7).mean()
        merge_df_scaled['rolling_mean_30'] = merge_df_scaled['sales'].rolling(window=30).mean()
        merge_df_scaled['rolling_mean_60'] = merge_df_scaled['sales'].rolling(window=60).mean()
        merge_df_scaled['rolling_mean_90'] = merge_df_scaled['sales'].rolling(window=90).mean()
        merge_df_scaled['rolling_mean_120'] = merge_df_scaled['sales'].rolling(window=120).mean()
    
        #rolling stdv
        merge_df_scaled['rolling_stdv_7'] = merge_df_scaled['sales'].rolling(window=7).std()
        merge_df_scaled['rolling_stdv_30'] = merge_df_scaled['sales'].rolling(window=30).std()
        merge_df_scaled['rolling_stdv_60'] = merge_df_scaled['sales'].rolling(window=60).std()
        merge_df_scaled['rolling_stdv_90'] = merge_df_scaled['sales'].rolling(window=90).std()
        merge_df_scaled['rolling_stdv_120'] = merge_df_scaled['sales'].rolling(window=120).std()
    
        merge_df_scaled.fillna(0,inplace=True)
        

    return merge_df_scaled


In [ ]:
def feature_extraction_transfer_test(train_df,test_df):

    # Ignore all warnings within this function
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
           
    
        #lagged features per years
        for i in range(1, 4):
            test_df[f'sales_lag_{i}years'] = train_df[f'sales_lag_{i}years'].iloc[-1]
    
            #rolling sum
        test_df['rolling_sum_7'] = train_df['rolling_sum_7'].iloc[-1]
        test_df['rolling_sum_30'] = train_df['rolling_sum_30'].iloc[-1]
        test_df['rolling_sum_60'] = train_df['rolling_sum_60'].iloc[-1]
        test_df['rolling_sum_90'] = train_df['rolling_sum_90'].iloc[-1]
        test_df['rolling_sum_120'] = train_df['rolling_sum_120'].iloc[-1]
        
        # Rolling average
        test_df['rolling_mean_7'] = train_df['rolling_mean_7'].iloc[-1]
        test_df['rolling_mean_30'] = train_df['rolling_mean_30'].iloc[-1]
        test_df['rolling_mean_60'] = train_df['rolling_mean_60'].iloc[-1]
        test_df['rolling_mean_90'] = train_df['rolling_mean_90'].iloc[-1]
        test_df['rolling_mean_120'] = train_df['rolling_mean_120'].iloc[-1]
        
        # Rolling standard deviation
        test_df['rolling_stdv_7'] = train_df['rolling_stdv_7'].iloc[-1]
        test_df['rolling_stdv_30'] = train_df['rolling_stdv_30'].iloc[-1]
        test_df['rolling_stdv_60'] = train_df['rolling_stdv_60'].iloc[-1]
        test_df['rolling_stdv_90'] = train_df['rolling_stdv_90'].iloc[-1]
        test_df['rolling_stdv_120'] = train_df['rolling_stdv_120'].iloc[-1]
    
        # Identify the last available date in the training data
        last_date_train = train_df.index[-1]
    
        # Fill in lagged features for the first few rows where future knowledge is available
        #for i in range(1, 8):
        #    # Identify the lagged date for the current lag
        #    lagged_date = last_date_train - pd.Timedelta(days=i)
            
            # Fill in the lagged sales values for corresponding lagged days from the training data
        #    test_df[f'sales_lag_{i}'] = test_df.index.map(lambda x: train_df.loc[x - pd.Timedelta(days=i), 'sales'] if x <= last_date_train else train_df[f'sales_lag_{i}'].iloc[-1])


        # Fill in lagged features for the first few rows where future knowledge is available
        for i in range(1, 8):
            test_df[f'sales_lag_{i}'] = np.nan  # Initialize with NaN
            
            # Iterate over each row in the test DataFrame
            for idx, row in test_df.iterrows():
                lagged_date = idx - pd.Timedelta(days=i)  # Calculate the lagged date
                
                # Check if the lagged date is within the training data range
                if lagged_date in train_df.index:
                    test_df.at[idx, f'sales_lag_{i}'] = train_df.loc[lagged_date, 'sales']
                else:
                    test_df.at[idx, f'sales_lag_{i}'] = train_df['sales'].iloc[-1]
    
    return test_df

In [ ]:
def calc_rmsse(train, test, predictions):
    forecast_mse = mean_squared_error(test, predictions)
    train_mse = ((train - train.shift(1)) ** 2).mean()
    return np.sqrt(forecast_mse / train_mse)

In [ ]:
def calculate_last_28_days_weights(df):
    most_recent_date = df.index.max()
    cutoff_date = most_recent_date - timedelta(days=27)
    last_28_days_df = df.loc[cutoff_date:most_recent_date]
    last_28_days_df['revenues'] = last_28_days_df['sales'] * last_28_days_df['sell_price']
    
    weights_df = last_28_days_df.groupby('id')['revenues'].sum()
    weights_df = pd.DataFrame(weights_df)
    
    total_revenues = last_28_days_df['revenues'].sum()
    weights_df['weights'] = weights_df['revenues'] / total_revenues
    
    return weights_df

In [ ]:
merge_df_scaled.drop(columns=["item_id","dept_id","state_id"],inplace=True)

In [ ]:
merge_df_scaled.fillna(0,inplace=True)

In [ ]:
merge_df_scaled["id"].unique()

# 1. Defining Model Functions

In [ ]:
def perform_prophet(product_data):

    product_data.reset_index(inplace=True,names="date")
    
    prophet_product_df = product_data[["id","date","sales"]]
    prophet_product_df.columns = ["id","ds","y"]
    prophet_product_df['ds'] = pd.to_datetime(prophet_product_df['ds'])
    
    data_train = prophet_product_df.iloc[:-28]
    data_test = prophet_product_df.iloc[-28:]
    X_train = data_train["ds"]
    y_train = data_train["y"]
    X_test = data_test["ds"]
    y_test = data_test["y"]
    
    fbp = Prophet()

    model = fbp.fit(data_train)
    
    predict_placeholder = fbp.make_future_dataframe(28,freq="D")
    
    # Predict on the test data
    y_pred = fbp.predict(predict_placeholder[-28:])
    

    # Calculate and return the error metric for the current fold
    rmsse = calc_rmsse(y_train, y_test, y_pred["yhat"])
    
    return model, rmsse

In [ ]:
def perform_auto_arima(product_data):
    data_train = product_data.iloc[:-28]
    data_test = product_data.iloc[-28:]
    y_train = data_train["sales"]
    y_test = data_test["sales"]

    # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
    model = auto_arima(y_train, start_p=0, start_q=0, max_p=5, max_q=5, d=1,
                       seasonal=True, trace=False, error_action='ignore', 
                       suppress_warnings=True, stepwise=True)
    
    # Predict on the test data
    predictions = model.predict(n_periods=len(y_test))

    # Calculate and return the error metric for the current fold
    rmsse = calc_rmsse(y_train,y_test,predictions)
    
    return model, rmsse

In [ ]:
def objective_optuna(trial, y_train, y_test):
    
    trend = trial.suggest_categorical('trend', ['add'])
    seasonal = trial.suggest_categorical('seasonal', [None, 'add'])
    seasonal_periods = trial.suggest_categorical('seasonal_periods', [None, 4, 7, 12])
    
    product_results = []

    # Fit Holt-Winters model on the training data
    model = ExponentialSmoothing(y_train, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods,freq='D')
    fitted_model = model.fit(optimized=True)

    # Predict on the test data
    predictions = fitted_model.forecast(steps=len(y_test))

    # Calculate and store the error metric
    mae = mean_absolute_error(y_test, predictions)
    

    # Calculate and return the error metric for the current fold
    rmsse = calc_rmsse(y_train,y_test, predictions)
    product_results.append(rmsse)

    # Average MAE for this product
    average_rmsse = np.mean(product_results)
    return average_rmsse

In [ ]:
def perform_exp_smoothing(product_data):
    data_train = product_data.iloc[:-28]
    data_test = product_data.iloc[-28:]
    y_train = data_train["sales"]
    y_test = data_test["sales"]
    # Create a study object
    study = optuna.create_study(direction='minimize')
    
    print(f"Optimizing hyperparameters for product: {id}")
    
    
    # Run the optimization process for the current product
    study.optimize(lambda trial: objective_optuna(trial, y_train, y_test), n_trials=10, n_jobs=-1)

    # Get the best hyperparameters and the corresponding best MAE
    best_params = study.best_params
    best_rmsse = study.best_value

    # Create the best model with the obtained hyperparameters
    best_model = ExponentialSmoothing(y_train, **best_params).fit()
    
    return best_model, best_rmsse

In [ ]:
def perform_lightgbm(product_data):
    
    data_train_val = product_data.iloc[:-28]
    data_train_val = feature_extraction(data_train_val)
    data_test = product_data.iloc[-28:]
    data_test = feature_extraction_transfer_test(data_train_val,data_test)

    data_train = data_train_val.iloc[:-112]
    data_val = data_train_val.iloc[-112:]
    
    X_train = data_train.drop(columns="sales")
    y_train = data_train["sales"]
    X_val = data_val.drop(columns="sales")
    y_val = data_val["sales"]
    X_test = data_test.drop(columns="sales")
    y_test = data_test["sales"]

    
    
    
    # Define LightGBM parameters
    params = {
        "n_estimators": 1000,
        'objective': 'regression',
        'metric': 'rmse',
        "boosting_type": "gbdt",
        "max_depth": -1,
        'learning_rate': 0.01,
        'feature_fraction': 0.4,
        "lambda_l1": 1,
        "lambda_l2": 1,
        "seed": 46,
    }

    model = lgb.LGBMRegressor(**params)
    
    # Create dataset for LightGBM
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    
    # Train the model
    num_round = 1000

    bst = lgb.train(params, lgb_train, num_round, valid_sets=lgb_eval, callbacks=[lgb.early_stopping(stopping_rounds=50)])
     
    # Make predictions for the next 28 days
    predictions = bst.predict(X_test, num_iteration=bst.best_iteration)

    # Calculate and return the error metric for the current fold
    rmsse = calc_rmsse(y_train,y_test, predictions)
    
    return bst, rmsse

# Example usage:
# sales_forecast = forecast_sales(product_data)
# print(sales_forecast)


In [ ]:
def prepare_data(product_data):
    target = product_data[['sales']]
    past_cov = product_data.drop(columns=['sales', 'id','item_id','dept_id','state_id','event_name_2'])
    future_cov = product_data.drop(columns=['sales','id','item_id','dept_id','state_id','event_name_2'])

    y_train = target.loc[:'2016-01-01']
    past_cov_train = past_cov.loc[:'2016-01-01']
    future_cov_train = future_cov.loc[:'2016-01-29']

    y_val = target.loc['2016-01-02':'2016-04-24']
    past_cov_val = past_cov.loc['2016-01-02':'2016-04-24']
    future_cov_val = future_cov.loc['2016-01-02':'2016-05-22']

    return (y_train, past_cov_train, future_cov_train,
            y_val, past_cov_val, future_cov_val)

In [ ]:
def train_tft_model(y_train_series, past_cov_train_series, future_cov_train_series,
                    y_val_series, past_cov_val_series, future_cov_val_series):
    input_chunk_length = 28*2
    output_chunk_length = 28

    from pytorch_lightning.callbacks.early_stopping import EarlyStopping

    my_stopper = EarlyStopping(
        monitor="val_loss",
        patience=30,
        min_delta=0.001,
        mode='min',
    )

    pl_trainer_kwargs={"callbacks": [my_stopper],
                       "accelerator": "cpu"}

    tft = TFTModel(input_chunk_length=input_chunk_length,
                   output_chunk_length=output_chunk_length,
                   pl_trainer_kwargs=pl_trainer_kwargs,
                   lstm_layers=2,
                   num_attention_heads=4,
                   dropout=0.2,
                   batch_size=16,
                   hidden_size=16,
                   torch_metrics=MeanAbsoluteError(),
                   n_epochs=100,
                   )

    tft.fit(series=y_train_series,
            past_covariates=past_cov_train_series,
            future_covariates=future_cov_train_series,
            val_series=y_val_series,
            val_past_covariates=past_cov_val_series,
            val_future_covariates=future_cov_val_series)

    return tft

#from darts.models import RNNModel

# model = TFTModel(input_chunk_length=28*2)

# model.save("my_model.pt")
# model_loaded = TFTModel.load("my_model.pt")

In [ ]:
def prepare_data_and_create_sequences(data, input_length, output_length, scaler=None):
    if scaler is None:
        scaler = MinMaxScaler(feature_range=(0, 1))
    data['scaled_sales'] = scaler.fit_transform(data[['sales']])
    
    X, y = [], []
    for i in range(len(data) - input_length - output_length + 1):
        X.append(data['scaled_sales'].values[i:(i + input_length)])
        y.append(data['scaled_sales'].values[(i + input_length):(i + input_length + output_length)])
    X = np.array(X).reshape((-1, input_length, 1))
    y = np.array(y)
    return X, y, scaler

def train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, input_length, output_length):
    n_features = X_train.shape[2]
    model = Sequential([LSTM(50, activation='relu'), Dense(output_length)])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    return model, mae

def run_lstm_pipeline(data, input_length, output_length, n_splits):
    fold_results = []
    tscv = TimeSeriesSplit(n_splits=n_splits)
    for train_index, test_index in tscv.split(data):
        train, test = data.iloc[train_index], data.iloc[test_index]

        # Prepare data and create sequences
        X_train, y_train, scaler = prepare_data_and_create_sequences(train, input_length, output_length)
        X_test, y_test, _ = prepare_data_and_create_sequences(test, input_length, output_length, scaler)

        # Train and evaluate model
        model, mae = train_and_evaluate_lstm_model(X_train, y_train, X_test, y_test, input_length, output_length)
        fold_results.append(mae)
    return fold_results

# 2.Running all models in a loop to find for each product with lowest score

In [ ]:
models_list = ["ARIMA","Prophet","ExponentialSmoothing","LightGBM","DartsTFT", "LSTM"]

In [ ]:
from pmdarima import auto_arima

# Dictionary to store MAE results for each unique time-series identified by id
product_results = {}
average_rmsse = []

ids_filtered=merge_df_scaled['id'].unique()
filtered_df = merge_df_scaled[merge_df_scaled['id'].isin(ids_filtered)]
weights=calculate_last_28_days_weights(filtered_df)   

# Iterate over each unique product series identified by id
for id in ids_filtered:
    print(f"Analyzing product: {id}")
    product_data = merge_df_scaled[merge_df_scaled['id'] == id].drop(columns="id")
    product_data_with_id = merge_df_scaled[merge_df_scaled['id'] == id]

    # Results list for the current product time-series
    results = {}
    best_score = 999.99
    best_model_name = ""


    product_weight=weights.loc[id].weights



    #Looping all models
    for model_name in models_list:

        if model_name == "ARIMA":
            #TODO: Add 5-fold split here for another loop (or inside the model function?) and then take the average score per model as their mae score
            
            # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
            model, rmsse = perform_auto_arima(product_data)
            results[model_name] = {"rmsse": rmsse, "model": model}
            if rmsse < best_score:
                best_score = rmsse
                best_model = model
                best_model_name = model_name

        elif model_name == "ExponentialSmoothing":

            # To be built
            model, rmsse = perform_exp_smoothing(product_data)
            results[model_name] = {"rmsse": rmsse, "model": model}
            if rmsse < best_score:
                best_score = rmsse
                best_model = model
                best_model_name = model_name

        elif model_name == "Prophet":

            model, rmsse = perform_prophet(product_data_with_id)
            results[model_name] = {"rmsse": rmsse, "model": model}
            if rmsse < best_score:
                best_score = rmsse
                best_model = model
                best_model_name = model_name


        elif model_name == "LightGBM":

            model, rmsse = perform_lightgbm(product_data)
            results[model_name] = {"rmsse": rmsse, "model": model}
            if rmsse < best_score:
                best_score = rmsse
                best_model = model
                best_model_name = model_name

        
        elif model_name == "DartsTFT":
            # Prepare data for TFT model
            (y_train, past_cov_train, future_cov_train,
             y_val, past_cov_val, future_cov_val) = prepare_data(product_data_with_id)

            # Example code assuming daily frequency ('D')
            y_train_series = TimeSeries.from_dataframe(y_train, fill_missing_dates=True, freq='D')
            past_cov_train_series = TimeSeries.from_dataframe(past_cov_train, fill_missing_dates=True, freq='D')
            future_cov_train_series = TimeSeries.from_dataframe(future_cov_train, fill_missing_dates=True, freq='D')

            y_val_series = TimeSeries.from_dataframe(y_val, fill_missing_dates=True, freq='D')
            past_cov_val_series = TimeSeries.from_dataframe(past_cov_val, fill_missing_dates=True, freq='D')
            future_cov_val_series = TimeSeries.from_dataframe(future_cov_val, fill_missing_dates=True, freq='D')

            trained_model = train_tft_model(y_train_series, past_cov_train_series, future_cov_train_series,
                                            y_val_series, past_cov_val_series, future_cov_val_series)

            # Store the trained TFT model in the results dictionary
            results[model_name] = {"rmsse": trained_model.validation_loss(), "model": trained_model}
            if trained_model.validation_loss() < best_score:
                best_score = trained_model.validation_loss()
                best_model = trained_model
                best_model_name = model_name

        elif model_name == "LSTM":
            input_length = 200
            output_length = 28
            n_splits = 10
            
            fold_results = run_lstm_pipeline(product_data, input_length, output_length, n_splits)
            mae = np.mean(fold_results)
            results[model_name] = {"mae": mae}
            if mae < best_score:
                best_score = mae
                best_model = None
                best_model_name = model_name

    
    #Printing results for this product
    print(results)
    print(f"Model results for {id}")
    print(f"Best model: {best_model_name}")
    print(f"Best score: {best_score}")

    average_rmsse.append(best_score*product_weight)

    # Store the average MAE for the current product time-series
    product_results[id] = {"best_score": best_score, "best_model": best_model_name, "model": best_model}

    #Store the best model in a pkl file
    if best_model_name == "DartsTFT":
        filename = f'../models/{id}_model.pt'
        best_model.save(filename)
    elif best_model_name == "LSTM":
        filename = f'../models/{id}_model.h5'
        best_model.save(filename)
    else:
        filename = f'../models/{id}_model.pkl'
        with open(filename, 'wb') as f:
            pickle.dump(best_model, f)
            
# Create a DataFrame to store the results
results_df_arima = pd.DataFrame(product_results.items(), columns=['id', 'RMSSE'])

# Set the 'id' column as the index
results_df_arima.set_index('id', inplace=True)

average_rmsse_score = np.sum(average_rmsse)

print(f"Total average wRMSSE: {average_rmsse_score}")
